# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 22

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28.0,1990.0,17,17,1444.0,2,0,1,1,0,0,0.12,0.00,0.12,0.06,0.06,1.5,0.8,0.9,0.09,0.06,0.15,0.05,0.10,Matches
2,Max Aarons\Max-Aarons,eng ENG,MF,Norwich City,19.0,2000.0,20,20,1800.0,0,0,0,0,5,0,0.00,0.00,0.00,0.00,0.00,0.4,0.4,3.1,0.02,0.15,0.17,0.02,0.17,Matches
3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21.0,1997.0,21,20,1673.0,13,3,0,0,2,0,0.70,0.16,0.86,0.70,0.86,10.0,10.0,2.2,0.54,0.12,0.65,0.54,0.65,Matches
4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23.0,1996.0,16,8,646.0,0,1,0,0,0,0,0.00,0.14,0.14,0.00,0.14,1.5,1.5,0.6,0.21,0.08,0.29,0.21,0.29,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,32.0,1987.0,10,8,783.0,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)

# change team names to match convention used in the FPL data
teamStats.loc[teamStats['Squad']=='Brighton & Hove Albion','Squad'] = 'Brighton'
teamStats.loc[teamStats['Squad']=='Manchester United','Squad'] = 'Manchester Utd'
teamStats.loc[teamStats['Squad']=='Newcastle United','Squad'] = 'Newcastle Utd'
teamStats.loc[teamStats['Squad']=='Sheffield United','Squad'] = 'Sheffield Utd'
teamStats.loc[teamStats['Squad']=='West Ham United','Squad'] = 'West Ham'
teamStats.loc[teamStats['Squad']=='Tottenham Hotspur','Squad'] = 'Tottenham'
teamStats.loc[teamStats['Squad']=='Wolverhampton Wanderers','Squad'] = 'Wolves'

teamStats.head()

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,
1,Liverpool,21,20,1,0,50,14,36,61,40.6,20.8,19.8,0.94,W W W W W,53118,Sadio Mané - 11,Alisson,NaN
2,Manchester City,22,15,2,5,62,25,37,47,52.2,22.1,30.1,1.37,W L W W W,54386,Sergio Agüero - 13,Ederson,NaN
3,Leicester City,22,14,3,5,47,21,26,45,35.0,27.3,7.7,0.35,L L W W L,32052,Jamie Vardy - 17,Kasper Schmeichel,NaN
4,Chelsea,22,12,3,7,39,29,10,39,37.9,21.8,16.1,0.73,W L W D W,40551,Tammy Abraham - 13,Kepa Arrizabalaga,NaN
5,Manchester Utd,22,9,7,6,36,25,11,34,34.6,20.7,13.9,0.63,L W W L W,72518,Marcus Rashford - 14,David de Gea,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)

# fetch data locally
#df = pd.read_csv('../data/data_week' + str(latest_gameweek) + '.csv', index_col=0)#,encoding='latin-1')

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,in_dreamteam,influence,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,35,100.0,100.0,0,69140,0,0,-3,3,1.2,0,2,0.8,0.8,0,Shkodran,0.3,4,0,8.8,False,32.2,170,,2019-11-28T23:00:21.541666Z,52,0,0,0,69140.jpg,2.0,0,0,Mustafi,0.3,False,None,a,1,3,54.0,6,8606,59,33498,179,0.1,1.2,Mustafi,0
2,0,0,26,25.0,25.0,0,98745,0,0,-1,1,22.7,0,2,0.1,0.1,0,Héctor,0.0,6,0,6.9,False,30.8,262,Hamstring injury - 25% chance of playing,2019-12-09T20:00:21.228098Z,54,0,0,0,98745.jpg,0.3,0,0,Bellerín,0.2,False,None,d,1,3,15.0,1,35058,28,37592,275,0.0,0.2,Bellerín,2
3,2,1,219,100.0,100.0,2,111457,0,0,-3,3,156.3,0,2,1.7,1.7,2,Sead,1.2,18,0,41.2,False,190.0,1086,,2019-12-15T19:30:19.136195Z,52,0,0,0,111457.jpg,2.1,0,0,Kolasinac,0.6,False,None,a,1,3,67.0,34,54384,1485,118168,522,0.2,6.5,Kolasinac,3
4,2,3,206,100.0,100.0,3,154043,0,0,-4,4,151.6,1,2,3.0,3.0,1,Ainsley,2.5,18,0,42.6,False,243.0,1120,,2019-09-22T18:00:10.824841Z,46,0,0,0,154043.jpg,2.6,1,0,Maitland-Niles,2.7,False,None,a,1,3,37.0,34,576919,3145,587244,2423,0.5,7.4,Maitland-Niles,3
5,0,5,286,100.0,100.0,3,39476,0,0,-1,1,31.8,1,2,3.7,3.7,2,Sokratis,3.2,25,2,57.1,False,431.6,1606,,2019-12-26T17:00:15.332949Z,49,0,0,0,39476.jpg,2.9,0,0,Papastathopoulos,1.5,False,None,a,1,3,106.0,52,132311,3030,162932,2138,0.7,10.6,Sokratis,5


## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [6]:
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xGA per game,probability no goals allowed
Rk,,,,,,,,,,,,,,,,,,,,
1,Liverpool,21,20,1,0,50,14,36,61,40.6,20.8,19.8,0.94,W W W W W,53118,Sadio Mané - 11,Alisson,NaN,0.990476,0.371400
2,Manchester City,22,15,2,5,62,25,37,47,52.2,22.1,30.1,1.37,W L W W W,54386,Sergio Agüero - 13,Ederson,NaN,1.004545,0.366211
3,Leicester City,22,14,3,5,47,21,26,45,35.0,27.3,7.7,0.35,L L W W L,32052,Jamie Vardy - 17,Kasper Schmeichel,NaN,1.240909,0.289121
4,Chelsea,22,12,3,7,39,29,10,39,37.9,21.8,16.1,0.73,W L W D W,40551,Tammy Abraham - 13,Kepa Arrizabalaga,NaN,0.990909,0.371239
5,Manchester Utd,22,9,7,6,36,25,11,34,34.6,20.7,13.9,0.63,L W W L W,72518,Marcus Rashford - 14,David de Gea,NaN,0.940909,0.390273
6,Sheffield Utd,22,8,8,6,24,21,3,32,26.9,24.8,2.1,0.09,W D L L W,30738,Lys Mousset - 5,Dean Henderson,NaN,1.127273,0.323915
7,Wolves,22,7,10,5,31,28,3,31,28.6,22.0,6.6,0.30,W W L L D,31313,Raúl Jiménez - 8,Rui Patrício,NaN,1.000000,0.367879
8,Tottenham,22,8,6,8,36,31,5,30,28.0,27.5,0.5,0.02,L W D L L,59464,Harry Kane - 11,Paulo Gazzaniga,NaN,1.250000,0.286505
9,Crystal Palace,22,7,8,7,20,24,-4,29,20.6,29.3,-8.7,-0.39,L W D D D,25095,Jordan Ayew - 6,Vicente Guaita,NaN,1.331818,0.263997


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

In [7]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  6 * (df.loc[ix, 'goals_scored'] - xG)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  5 * (df.loc[ix, 'goals_scored'] - xG)
    elif df.loc[ix, 'element_type']==4:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  4 * (df.loc[ix, 'goals_scored'] - xG)

In [8]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[teamStats['Squad']==team, 'probability no goals allowed'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    4 * (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players assists based on their xA.

In [9]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)

In [10]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

In [11]:
team_names = np.sort(playerStats['Squad'].unique())
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
#df['games played'] = df['minutes']/90.0
xA_proportion

0.756060606060606

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [12]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','garcia','chicharito','elliott']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'chicharito':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('javierhernandez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [13]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

In [14]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
191,Salah,18.000000,126,7.0,120.744287,6.708016
192,Mané,19.861111,143,7.2,129.330958,6.511769
281,McGovern,2.000000,11,5.5,12.593297,6.296648
215,De Bruyne,21.126761,150,7.1,129.745944,6.141308
166,Vardy,19.866667,149,7.5,120.790909,6.080079
233,Rashford,21.967213,134,6.1,129.159091,5.879630
214,Sterling,20.000000,110,5.5,113.487857,5.674393
182,Alexander-Arnold,21.071429,118,5.6,117.521879,5.577309
210,Agüero,15.937500,102,6.4,83.527273,5.240927


In [15]:
# save data
filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [16]:
playerStats['xG'].sum()

590.5

In [17]:
df['goals_scored'].sum()

593

In [18]:
playerStats['xG'].sum()/df['goals_scored'].sum()

0.9957841483979764